<a href="https://colab.research.google.com/github/sayakpaul/Training-BatchNorm-and-Only-BatchNorm/blob/master/All_Layers_Frozen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Authenticate yourself to use the TPUs
import os

IS_COLAB_BACKEND = 'COLAB_GPU' in os.environ  # this is always set on Colab, the value is 0 or 1 depending on GPU presence
if IS_COLAB_BACKEND:
  from google.colab import auth
  # Authenticates the Colab machine and also the TPU using your
  # credentials so that they can access your private GCS buckets.
  auth.authenticate_user()

In [2]:
# TensorFlow Imports
import tensorflow as tf
print(tf.__version__)

2.2.0


In [0]:
# Detect hardware
try:
  tpu = tf.distribute.cluster_resolver.TPUClusterResolver() # TPU detection
except ValueError:
  tpu = None
  gpus = tf.config.experimental.list_logical_devices("GPU")
    
# Select appropriate distribution strategy
if tpu:
  tf.config.experimental_connect_to_cluster(tpu)
  tf.tpu.experimental.initialize_tpu_system(tpu)
  strategy = tf.distribute.experimental.TPUStrategy(tpu) # Going back and forth between TPU and host is expensive. Better to run 128 batches on the TPU before reporting back.
  print('Running on TPU ', tpu.cluster_spec().as_dict()['worker'])  
elif len(gpus) > 1:
  strategy = tf.distribute.MirroredStrategy([gpu.name for gpu in gpus])
  print('Running on multiple GPUs ', [gpu.name for gpu in gpus])
elif len(gpus) == 1:
  strategy = tf.distribute.get_strategy() # default strategy that works on CPU and single GPU
  print('Running on single GPU ', gpus[0].name)
else:
  strategy = tf.distribute.get_strategy() # default strategy that works on CPU and single GPU
  print('Running on CPU')
print("Number of accelerators: ", strategy.num_replicas_in_sync)

In [0]:
!pip install wandb -q
import wandb
wandb.login()

In [0]:
!wget https://raw.githubusercontent.com/GoogleCloudPlatform/keras-idiomatic-programmer/master/zoo/resnet/resnet_cifar10.py

In [0]:
# Other imports
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from wandb.keras import WandbCallback
import matplotlib.pyplot as plt
import tensorflow as tf
import resnet_cifar10
import numpy as np
import time

# Random seed fixation
tf.random.set_seed(666)
np.random.seed(666)

In [0]:
def get_training_model():
    # ResNet20
    n = 2
    depth =  n * 9 + 2
    n_blocks = ((depth - 2) // 9) - 1

    # The input tensor
    inputs = Input(shape=(32, 32, 3))

    # The Stem Convolution Group
    x = resnet_cifar10.stem(inputs)

    # The learner
    x = resnet_cifar10.learner(x, n_blocks)

    # The Classifier for 10 classes
    outputs = resnet_cifar10.classifier(x, 10)

    # Instantiate the Model
    model = Model(inputs, outputs)

    model.compile(loss="sparse_categorical_crossentropy", optimizer="sgd", metrics=["accuracy"])
    
    return model

In [8]:
# Load the training set of CIFAR10
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

170500096/170498071 [==============================] - 2s 0us/step


In [0]:
BATCH_SIZE = 128 * strategy.num_replicas_in_sync

def normalize(image, label):
    image = tf.image.convert_image_dtype(image, tf.float32)
    label = tf.cast(label, tf.int32)
    return image, label

def augment(image,label):
    image = tf.image.resize_with_crop_or_pad(image, 40, 40) # Add 8 pixels of padding
    image = tf.image.random_crop(image, size=[32, 32, 3]) # Random crop back to 28x28
    image = tf.image.random_brightness(image, max_delta=0.5) # Random brightness
    image = tf.clip_by_value(image, 0., 1.)

    return image, label

train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_ds = (
    train_ds
    .shuffle(1024)
    .map(normalize, num_parallel_calls=tf.data.experimental.AUTOTUNE)
    .map(augment, num_parallel_calls=tf.data.experimental.AUTOTUNE)
    .batch(BATCH_SIZE)
    .prefetch(tf.data.experimental.AUTOTUNE)
)

test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test))
test_ds = (
    test_ds
    .map(normalize, num_parallel_calls=tf.data.experimental.AUTOTUNE)
    .batch(BATCH_SIZE)
    .prefetch(tf.data.experimental.AUTOTUNE)
)

In [13]:
# Set all the layers to *non-trainable*
with strategy.scope():
    model = get_training_model()
    for layer in model.layers:
        layer.trainable = False

    model.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

len(model.trainable_variables)

0

In [14]:
# Train model with a decay schedule
wandb.init(project="training-bn-only", id="all-layers-frozen")

start = time.time()
h = model.fit(train_ds,
         validation_data=test_ds,
         epochs=75,
         callbacks=[WandbCallback()])
end = time.time()
wandb.log({"training_time": end - start})
print("Network takes {:.3f} seconds to train".format(end - start))

Epoch 1/75
49/49 [==============================] - 7s 152ms/step - accuracy: 0.1000 - loss: 9.2736 - val_accuracy: 0.1000 - val_loss: 9.9058
Epoch 2/75
49/49 [==============================] - 2s 50ms/step - accuracy: 0.1000 - loss: 9.2854 - val_accuracy: 0.1000 - val_loss: 9.9058
Epoch 3/75
49/49 [==============================] - 2s 49ms/step - accuracy: 0.1000 - loss: 9.2765 - val_accuracy: 0.1000 - val_loss: 9.9058
Epoch 4/75
49/49 [==============================] - 3s 56ms/step - accuracy: 0.1000 - loss: 9.2775 - val_accuracy: 0.1000 - val_loss: 9.9058
Epoch 5/75
49/49 [==============================] - 2s 48ms/step - accuracy: 0.1000 - loss: 9.2695 - val_accuracy: 0.1000 - val_loss: 9.9058
Epoch 6/75
49/49 [==============================] - 2s 47ms/step - accuracy: 0.1000 - loss: 9.2926 - val_accuracy: 0.1000 - val_loss: 9.9058
Epoch 7/75
49/49 [==============================] - 2s 47ms/step - accuracy: 0.1000 - loss: 9.2763 - val_accuracy: 0.1000 - val_loss: 9.9058
Epoch 8/75
4